# Tutorial 5: Your First AI Agent

**Difficulty:** Beginner | **Time:** 25 minutes

## Learning Objectives

- Understand the difference between Nodes and Agents
- Create and configure an Agent
- Work with different LLM providers
- Handle agent inputs and outputs

## Real-World Use Case

Imagine you want to build an AI assistant that can answer questions, help with customer service, or analyze documents. While you could build this using regular nodes with LLM calls, Spark provides a specialized **Agent** class that handles the complexity of AI interactions for you.

In this tutorial, you'll learn how to create AI agents that can:
- Answer questions using different language models
- Maintain context and conversations
- Use structured prompts and system instructions
- Integrate with multiple AI providers (OpenAI, Bedrock, Ollama, etc.)

We'll start by building a simple LLM node to understand the fundamentals, then move to the more powerful Agent class.

## Core Concepts

### Node vs Agent

**Node** (from Tutorial 1):
- Generic processing unit
- You define exactly what it does in `process()`
- Manual handling of inputs/outputs
- Good for data transformation, calculations, workflows

**Agent** (new in this tutorial):
- Specialized for AI/LLM interactions
- Handles LLM calls, message formatting, and responses automatically
- Built-in support for conversations, tools, and structured output
- Ideal for AI assistants, chatbots, and reasoning tasks

### When to Use Each

Use **Nodes** for:
- Data processing and transformations
- API integrations (non-LLM)
- Workflow orchestration
- When you need full control over the logic

Use **Agents** for:
- Chatbots and conversational AI
- Question answering and reasoning
- Content generation and analysis
- When working with language models

## Setup

Let's import the necessary classes and set up our environment.

In [1]:
# Import the classes we need
from spark.nodes import Node
from spark.agents.agent import Agent
from spark.models import OpenAIModel
from spark.agents.config import AgentConfig
from spark.utils.common import ask_llm, arun

# For testing without API keys
from spark.models.echo import EchoModel

import asyncio
import time

## Example 1: LLM Node (The Foundation)

Before we jump to Agents, let's build a simple node that calls an LLM. This will help you understand what's happening under the hood when we use Agents.

### Key Points:
- Use `ask_llm()` utility for simple LLM calls
- You handle the input/output formatting manually
- Good for understanding the basics but limited for complex interactions

In [4]:
class SimpleLLMNode(Node):
    """A node that calls an LLM to answer questions."""
    
    def process(self, context):
        # Extract the question from inputs
        question = context.inputs.content.get('question', '')
        
        if not question:
            return {'error': 'No question provided'}
        
        print(f"🤔 Question: {question}")
        
        # Use the ask_llm utility to get an answer
        answer = ask_llm(question)
        
        return {
            'question': question,
            'answer': answer,
            'timestamp': time.time()
        }

In [5]:
# Test the Simple LLM Node
llm_node = SimpleLLMNode()

# Ask some questions
questions = [
    "What is the capital of France?",
    "Explain photosynthesis in one sentence.",
    "Why is the sky blue? Explain in under 150 words"
]

print("=== Simple LLM Node Demo ===")
for question in questions:
    result = await llm_node.run({'question': question})
    print(f"✅ Answer: {result.content['answer']}\n")

=== Simple LLM Node Demo ===
🤔 Question: What is the capital of France?
✅ Result: Paris.

🤔 Question: Explain photosynthesis in one sentence.
✅ Result: Photosynthesis is the process by which green plants, algae, and some bacteria use light energy to convert carbon dioxide and water into glucose and oxygen.

🤔 Question: Why is the sky blue? Explain in under 150 words
✅ Result: Sunlight is white, made of many colors. When it hits Earth’s atmosphere, it collides with gas molecules and scatters. Shorter wavelengths (blue and violet) scatter much more than longer ones (red, yellow). This Rayleigh scattering makes blue light spread in all directions. Our eyes see blue from every part of the sky, so it looks blue.

Violet light scatters even more, but the sky isn’t violet because the atmosphere absorbs much of it and our eyes are less sensitive to violet.

At sunrise and sunset, sunlight travels through more air, scattering away the blue and green colors and leaving reds and oranges, which is

### Understanding the ask_llm() Utility

The `ask_llm()` utility is a simple wrapper that:
1. Takes a text prompt
2. Sends it to a configured LLM (defaults to OpenAi gpt-5-mini for testing)
3. Returns the text response

**Other Models**: To use real LLMs, set environment variables:
- `OPENAI_API_KEY` for OpenAI models

## Example 2: Enter the Agent

Now let's use Spark's Agent class, which provides much more powerful and flexible AI interactions.

### Key Advantages of Agents:
- **Message-based interaction**: Proper conversation handling
- **Multiple models**: Easy switching between OpenAI, Bedrock, Ollama, etc.
- **Configuration**: System prompts, templates, and behavior control
- **Extensibility**: Built-in support for tools, memory, and structured output
- **Error handling**: Robust error management and retry logic

In [6]:
# Create a simple Agent with EchoModel (for testing - no API key needed)
agent = Agent(model_id='openai/gpt-5-mini')

print("=== Simple Agent Demo ===")
question = "What is the matthew effect in under 150 words?"
print(f"Question: {question}")

# Execute the agent
await agent.run({'messages': [{"role": "user", "content": question}]})

print(f"Answer: {agent.outputs and agent.outputs.content}")

=== Simple Agent Demo ===
Question: What is the matthew effect in under 150 words?
Answer: The Matthew effect is the sociological phenomenon where advantages accumulate to those who already have them—“the rich get richer.” Coined by Robert K. Merton (1968) after the Gospel of Matthew, it describes how small initial differences in status, recognition, resources, or success lead to growing disparities because benefits (funding, attention, opportunities) concentrate with the already successful. It shows up in science (well-known researchers get more credit), education (early achievers get more support), economics, and online networks (popular content gains more visibility).


### Understanding Agent Messages

Agents expect messages in a specific format:

```python
messages = [
    {"role": "user", "content": "Your question here"},
    {"role": "assistant", "content": "Previous response"},
    {"role": "user", "content": "Follow-up question"}
]
```

This format allows agents to maintain conversation context and handle multi-turn dialogues.

## Example 3: Agent with Configuration

Let's create a more sophisticated agent using AgentConfig. This gives us control over the agent's behavior, personality, and capabilities.

In [7]:
# Create an agent with configuration
qa_agent = Agent(
    config=AgentConfig(
        model=OpenAIModel(model_id='gpt-5-mini'),
        system_prompt="""You are a helpful AI assistant. You are knowledgeable, friendly, and provide clear, concise answers.
        
        Guidelines:
        - Answer questions directly and accurately
        - If you don't know something, admit it
        - Keep responses under 100 words when possible
        - Be encouraging and positive""",
        prompt_template="User Question: {{question}}"
    )
)

print("=== Configured Agent Demo ===")

# Test with different types of questions
test_questions = [
    "What is machine learning?",
    "How do I bake a chocolate cake?",
    "Explain the concept of recursion in programming."
]

for question in test_questions:
    print(f"\n🙋‍♀️ Question: {question}")
    
    # Execute with the question
    await qa_agent.do({
        'messages': [{"role": "user", "content": question}]
    })
    
    answer = qa_agent.outputs and qa_agent.outputs.content
    print(f"🤖 Answer: {answer}")

=== Configured Agent Demo ===

🙋‍♀️ Question: What is machine learning?
🤖 Answer: Machine learning is a branch of artificial intelligence where computers learn patterns from data to make predictions or decisions without being explicitly programmed for each task. A model is trained on labeled or unlabeled data (supervised, unsupervised, or reinforcement learning), then evaluated and used to predict new cases. Common uses include spam filters, recommendations, image recognition, and fraud detection. Want a simple example or a deeper dive into one type?

🙋‍♀️ Question: How do I bake a chocolate cake?
🤖 Answer: Simple chocolate cake

Ingredients:
- 1 3/4 cups (220 g) all‑purpose flour
- 3/4 cup (75 g) unsweetened cocoa powder
- 2 cups (400 g) sugar
- 1 1/2 tsp baking powder, 1 1/2 tsp baking soda, 1 tsp salt
- 2 large eggs
- 1 cup (240 ml) milk
- 1/2 cup (120 ml) vegetable oil
- 2 tsp vanilla
- 1 cup (240 ml) boiling water (or hot coffee)

Instructions:
1. Preheat oven to 350°F (175°C). Gr

### Understanding AgentConfig Options

The `AgentConfig` we used includes:

- **`model_id`**: Which LLM to use ('echo' for testing, 'openai/gpt-4o-mini', etc.)
- **`system_prompt`**: Instructions that define the agent's behavior and personality
- **`prompt_template`**: Jinja2 template for formatting user inputs

We'll explore more configuration options in future tutorials (tools, memory, structured output, etc.).

## Example 4: Different Model Providers

One of the powerful features of Spark Agents is the ability to switch between different LLM providers. Let's explore how to work with various models.

### Available Providers:
- **EchoModel**: Testing (no API key)
- **OpenAI**: GPT models (requires OPENAI_API_KEY, if you use OpenAI compatible models, provide a base_url)
- **Bedrock**: AWS models (requires AWS credentials)
- **Gemini**: Google models (requires GEMINI_API_KEY)
- **Others**: More will be added in the future

To use real models (not just the echo model), you need to configure API keys:

```bash
# For OpenAI
export OPENAI_API_KEY="your-openai-api-key"

# For AWS Bedrock
export AWS_ACCESS_KEY_ID="your-access-key"
export AWS_SECRET_ACCESS_KEY="your-secret-key"
export AWS_REGION="us-east-1"

# For Google Gemini
export GEMINI_API_KEY="your-gemini-api-key"

agent = Agent(model_id='openai/gpt-4o-mini')
or
agent = Agent(model_id='bedrock/us.anthropic.claude-sonnet-4-5-20250929-v1:0')


## Example 5: Building a Practical Q&A Agent

Let's build a practical Q&A agent that demonstrates the key concepts we've learned. This agent will be designed to answer educational questions in a specific style.

In [14]:
tutor = Agent(
    config=AgentConfig(
        model=OpenAIModel(model_id='gpt-5-mini'),
        system_prompt="""You are an enthusiastic and patient educational tutor. Your goal is to help students learn concepts clearly.
                
            Your teaching style:
            - Start with a simple, direct answer
            - Provide one concrete example or analogy
            - End with an encouraging follow-up question
            - Keep explanations under 150 words
            - Use emojis to make learning fun 🎓✨
        """,
        prompt_template="Student Question: {{question}}\n\nSubject: {{subject}}"
    )
)

print("=== Educational Q&A Agent Demo ===")

educational_questions = [
    {"question": "What is photosynthesis?", "subject": "Biology"},
    {"question": "Why do we need to learn math?", "subject": "Mathematics"},
    {"question": "What caused World War I?", "subject": "History"},
    {"question": "How do computers work?", "subject": "Computer Science"}
]

for item in educational_questions:    
    print(f"\n📚 Subject: {item['subject']}")
    print(f"🙋‍♀️ Question: {item['question']}")
    
    answer = await tutor.run(item)
    print(f"🎓 Tutor: {answer.content}")

=== Educational Q&A Agent Demo ===

📚 Subject: Biology
🙋‍♀️ Question: What is photosynthesis?
🎓 Tutor: Simple answer: Photosynthesis is the process plants, algae, and some bacteria use to turn sunlight, carbon dioxide (CO2), and water (H2O) into sugar (glucose) and oxygen (O2) — it’s how they make food and store energy. 🌿✨

How it works in one line: chlorophyll captures light energy, uses it to split water (releasing O2) and make energy carriers (ATP, NADPH), then those power the Calvin cycle to fix CO2 into glucose.

Analogy: Think of a leaf as a solar-powered factory — sunlight is the electricity, chlorophyll are the solar panels, and the factory builds sugar “batteries” for the plant. ⚡➡️🍬

Want a short diagram or a quick walk-through of the light and dark (Calvin cycle) steps? 🎓

📚 Subject: Mathematics
🙋‍♀️ Question: Why do we need to learn math?
🎓 Tutor: Simple answer: We learn math because it teaches clear thinking and problem-solving we use every day — from managing money and un

## Example 6: Comparison - Node vs Agent

Let's compare our SimpleLLMNode with our Agent to understand the differences in practice.

In [16]:
# Create both implementations
simple_node = SimpleLLMNode()
simple_agent = Agent(model_id='openai/gpt-5-nano')

comparison_question = "Explain the difference between weather and climate."

print("=== Node vs Agent Comparison ===")
print(f"Question: {comparison_question}\n")

# Test the Node approach
print("1️⃣ Simple LLM Node:")
node_result = await simple_node.run({'question': comparison_question})
node_answer = node_result.content['answer']
print(f"   Answer: {node_answer}")
print(f"   Raw output: {node_result.content}")

print("\n" + "="*50 + "\n")

# Test the Agent approach
print("2️⃣ Agent:")
agent_outputs = await simple_agent.run(comparison_question)
agent_answer = agent_outputs.content
print(f"   Answer: {agent_answer}")
print(f"   Agent outputs: {simple_agent.outputs}")

=== Node vs Agent Comparison ===
Question: Explain the difference between weather and climate.

1️⃣ Simple LLM Node:
🤔 Question: Explain the difference between weather and climate.
   Answer: - Weather: The current state of the atmosphere at a specific place and time. It includes temperature, humidity, precipitation, wind, clouds, etc. Example: “It’s 28°C with light rain in Tokyo at 3 PM,” or a forecast for the next 24–72 hours.

- Climate: The long-term patterns of weather in a region, described by averages and ranges over many years (usually about 30 years). It includes typical temperatures, precipitation, seasons, and how often extreme events occur. Example: “Seattle has a wet climate with mild winters and light summers,” or “the 30-year normal rainfall for this area is about 100 cm per year.”

Key differences:
- Timescale: Weather is short-term (minutes to days); climate is long-term (years to decades).
- What’s described: Weather describes current conditions; climate describes ave

### Key Differences Observed

1. **Input Format**: 
   - Node: `{'question': 'text'}`
   - Agent: `{'messages': [{'role': 'user', 'content': 'text'}]}`
     When the argument is a string, it will be automatically converted to Messages.

2. **Output Format**:
   - Node: Standardized NodeMessage with content
   - Agent: same

3. **Extensibility**:
   - Node: Limited to what we implement in `process()`
   - Agent: Built-in support for conversations, tools, memory, etc.

4. **Configuration**:
   - Node: Hardcoded behavior
   - Agent: Configurable via AgentConfig (system prompts, templates, tools, etc.)

## Key Takeaways

### ✅ What You Learned:

1. **Nodes vs Agents**:
   - Nodes: Generic processing units with full control
   - Agents: Specialized for AI/LLM interactions with built-in features

2. **Simple LLM Calls**:
   - Use `ask_llm()` utility for basic LLM interactions
   - Good for understanding fundamentals but limited functionality

3. **Agent Fundamentals**:
   - Create agents with `Agent(model_id='...')`
   - Use message format for conversations
   - Access results via `agent.outputs.content`

4. **Agent Configuration**:
   - Use `AgentConfig` to customize behavior
   - Set system prompts for personality and instructions
   - Use prompt templates for input formatting

5. **Model Providers**:
   - Easy switching between OpenAI, Bedrock, Ollama, etc.
   - Just change `model_id` parameter
   - EchoModel available for testing without API keys

6. **Practical Applications**:
   - Q&A systems
   - Educational tutors
   - Customer service assistants
   - Content generation and analysis

## 💪 Practice Exercises

Test your understanding with these hands-on exercises!

### Exercise 1: Create a Specialized Agent

Create a `CodeReviewAgent` that provides feedback on Python code. The agent should:
- Be encouraging and constructive
- Focus on one improvement area at a time
- Suggest specific code changes
- Keep feedback under 100 words

In [ ]:
# Exercise 1: Create a Code Review Agent
class CodeReviewAgent:
    """Your code here!"""
    
    def __init__(self, model_id='echo'):
        # TODO: Create an Agent with appropriate configuration
        pass
    
    async def review_code(self, code):
        # TODO: Implement code review functionality
        pass

# Test your agent
# reviewer = CodeReviewAgent()
# test_code = """
# def calculate_total(items):
#     total = 0
#     for item in items:
#         total += item
#     return total
# """
# feedback = await reviewer.review_code(test_code)
# print(f"Code Review Feedback: {feedback}")

### Exercise 2: Build a Translation Agent

Create an agent that translates text between languages. Include error handling for unsupported languages.

In [ ]:
# Exercise 2: Create a Translation Agent
class TranslationAgent:
    """Your code here!"""
    
    def __init__(self, model_id='echo'):
        # TODO: Create an Agent for translation
        pass
    
    async def translate(self, text, from_lang, to_lang):
        # TODO: Implement translation with error handling
        pass

# Test your agent
# translator = TranslationAgent()
# result = await translator.translate("Hello world", "English", "Spanish")
# print(f"Translation result: {result}")

### Exercise 3: Multi-Turn Conversation

Create a simple chatbot that maintains conversation context across multiple turns.

In [ ]:
# Exercise 3: Create a Conversational Agent
class ConversationalAgent:
    """Your code here!"""
    
    def __init__(self, model_id='echo'):
        # TODO: Create an Agent and maintain conversation history
        pass
    
    async def chat(self, message):
        # TODO: Handle multi-turn conversation
        pass

# Test your conversational agent
# chatbot = ConversationalAgent()
# await chatbot.chat("Hi, my name is Alice")
# await chatbot.chat("What did I just tell you?")
# await chatbot.chat("Can you recommend a good book?")

## ✅ Solutions

Try the exercises yourself first! Solutions are provided below.

In [19]:
# Solution 1: Code Review Agent
code_review_agent = Agent(
    config=AgentConfig(
        model=OpenAIModel(model_id='gpt-5-mini'),
        system_prompt="""You are a supportive code reviewer. Your goal is to help developers improve their skills.
                
            Review guidelines:
            - Always start with something positive
            - Focus on ONE improvement opportunity per review
            - Provide specific, actionable suggestions
            - Include a code example when helpful
            - Keep feedback under 100 words
            - End with encouragement 🚀
        """,
        prompt_template="Please review this Python code:\n\n```python\n{code}\n```"
    )
)

test_code = """
def calculate_total(items):
    total = 0
    for item in items:
        total += item
    return total
"""
feedback = await code_review_agent.run({"code": test_code})
print(f"💻 Code Review Feedback: {feedback.content}")

💻 Code Review Feedback: Great that you want feedback — thanks for sharing your work! 

One key improvement: include the actual code and a minimal reproducible example (what it should do, what it does, and any errors). Actionable: paste the function/class plus sample inputs and expected vs actual outputs.

Example:
```python
def my_func(x):
    return x*2

# input: 3
# expected: 6
# actual: 5  # paste observed behavior/errors here
```
Send that and I’ll review the logic/style/security quickly. 🚀


In [20]:
# Solution 2: Translation Agent
class TranslationAgent:
    """An agent that translates text between languages."""
    
    def __init__(self):
        self.agent = Agent(
            config=AgentConfig(
                model=OpenAIModel(model_id='gpt-5-mini'),
                system_prompt="""You are a professional translator. You provide accurate translations while preserving meaning and tone.
                
                Translation rules:
                - Translate accurately and naturally
                - Preserve the original tone and intent
                - If you don't know a language, say so politely
                - For unsupported languages, suggest alternatives
                - Keep the translation concise and clear""",
                prompt_template="Translate the following text from {from_lang} to {to_lang}:\n\nText: {text}"
            )
        )
        
        # Common languages for validation
        self.supported_languages = {
            'english', 'spanish', 'french', 'german', 'italian', 
            'portuguese', 'chinese', 'japanese', 'korean', 'russian',
            'arabic', 'hindi', 'dutch', 'swedish', 'norwegian'
        }
    
    async def translate(self, text, from_lang, to_lang):
        # Validate inputs
        if not text or not text.strip():
            return "Please provide text to translate."
        
        from_lang = from_lang.lower()
        to_lang = to_lang.lower()
        
        # Check language support
        if from_lang not in self.supported_languages:
            return f"Sorry, I don't support translating from {from_lang}. Supported languages: {', '.join(sorted(self.supported_languages))}"
        
        if to_lang not in self.supported_languages:
            return f"Sorry, I don't support translating to {to_lang}. Supported languages: {', '.join(sorted(self.supported_languages))}"
        
        # Don't translate if same language
        if from_lang == to_lang:
            return "The source and target languages are the same. No translation needed."
        
        formatted_prompt = f"Translate the following text from {from_lang} to {to_lang}:\n\nText: {text}"
        
        await self.agent.do({
            'messages': [{"role": "user", "content": formatted_prompt}]
        })
        
        return self.agent.outputs and self.agent.outputs.content

# Test the solution
translator = TranslationAgent()
result = await translator.translate("Hello world", "english", "spanish")
print(f"🌍 Translation result: {result}")

# Test error handling
error_result = await translator.translate("Hello", "english", "klingon")
print(f"❌ Error handling: {error_result}")

🌍 Translation result: Hola, mundo
❌ Error handling: Sorry, I don't support translating to klingon. Supported languages: arabic, chinese, dutch, english, french, german, hindi, italian, japanese, korean, norwegian, portuguese, russian, spanish, swedish


In [ ]:
# Solution 3: Conversational Agent
class ConversationalAgent:
    """An agent that maintains conversation context across multiple turns."""
    
    def __init__(self):
        self.agent = Agent(
            config=AgentConfig(
                model=OpenAIModel(model_id='gpt-5-mini'),
                system_prompt="""You are a friendly and helpful AI assistant. You:
                - Remember previous parts of the conversation
                - Reference earlier messages when relevant
                - Maintain a consistent, helpful personality
                - Ask follow-up questions when appropriate
                - Keep responses conversational but concise"""
            )
        )
        self.conversation_history = []
    
    async def chat(self, message):
        if not message or not message.strip():
            return "Please say something!"
        
        # Add user message to history
        self.conversation_history.append({
            "role": "user", 
            "content": message
        })
        
        # Send the entire conversation history
        await self.agent.do({
            'messages': self.conversation_history
        })
        
        # Get agent response
        response = self.agent.outputs and self.agent.outputs.content
        
        # Add agent response to history
        if response:
            self.conversation_history.append({
                "role": "assistant", 
                "content": response
            })
        
        return response
    
    def get_conversation_summary(self):
        """Return a summary of the conversation so far."""
        total_messages = len(self.conversation_history)
        user_messages = len([m for m in self.conversation_history if m["role"] == "user"])
        assistant_messages = len([m for m in self.conversation_history if m["role"] == "assistant"])
        
        return {
            "total_messages": total_messages,
            "user_messages": user_messages,
            "assistant_messages": assistant_messages,
            "last_topic": self.conversation_history[-1]["content"] if self.conversation_history else "No conversation yet"
        }

# Test the solution
chatbot = ConversationalAgent()

print("💬 Starting conversation...")

# Simulate a conversation
messages = [
    "Hi, my name is Alice and I love hiking.",
    "What did I just tell you about myself?",
    "Can you recommend a good hiking trail near San Francisco?",
    "That sounds great! Do you remember my name?"
]

for i, message in enumerate(messages, 1):
    print(f"\n--- Turn {i} ---")
    print(f"🙋‍♀️ User: {message}")
    
    response = await chatbot.chat(message)
    print(f"🤖 Assistant: {response}")

print("\n" + "="*50)
summary = chatbot.get_conversation_summary()
print(f"📊 Conversation Summary: {summary}")

💬 Starting conversation...

--- Turn 1 ---
🙋‍♀️ User: Hi, my name is Alice and I love hiking.
🤖 Assistant: Hi Alice — nice to meet you! I’m glad you love hiking — me too. Do you have favorite trails or a region you usually hike in?

If you want, I can:
- Suggest trails near you
- Make a packing checklist for day hikes or overnight trips
- Build a training plan to get fitter for longer hikes
- Give gear recommendations or safety tips

Quick tips to get started: check the weather and trail conditions, wear layered clothing and sturdy footwear, carry enough water and a map/phone with offline maps, and follow Leave No Trace. Which of the options above would you like first?

--- Turn 2 ---
🙋‍♀️ User: What did I just tell you about myself?
🤖 Assistant: You told me your name is Alice and that you love hiking. Would you like trail suggestions, a packing checklist, or something else related to hiking?

--- Turn 3 ---
🙋‍♀️ User: Can you recommend a good hiking trail near San Francisco?


---

## 🎯 Summary & Next Steps

### Congratulations! 🎉

You've successfully learned how to create and work with AI Agents in Spark ADK! You've built a solid foundation for creating intelligent, conversational AI systems.

### What You Mastered:

✅ **Understanding Nodes vs Agents**: When to use each approach

✅ **Simple LLM Calls**: Using `ask_llm()` for basic AI interactions

✅ **Agent Creation**: Building agents with different model providers

✅ **Agent Configuration**: Using AgentConfig for customization

✅ **Message Format**: Proper conversation handling with role-based messages

✅ **Practical Applications**: Q&A systems, educational tutors, code reviewers

✅ **Error Handling**: Robust agent design with input validation

✅ **Multi-turn Conversations**: Maintaining context across interactions

### 📚 Related Documentation:
- Example files: `e002_single_llm_call_node.py`, `e003_single_agent.py`
- Source: `spark/agents/agent.py`, `spark/agents/config.py`
- Models: `spark/models/` directory for different providers

### 🚀 Next Tutorial: Conditional Routing and Decision Making

In **Tutorial 5**, you'll learn how to:
- Create conditional edges between nodes
- Route based on agent outputs and decisions
- Build decision trees and branching workflows
- Implement loops and cycles in graphs
- Use LLMs to make routing decisions

### 🔧 Before You Move On:

Make sure you can:
1. ✅ Create both simple LLM nodes and agents
2. ✅ Configure agents with system prompts and templates
3. ✅ Switch between different model providers
4. ✅ Handle errors and edge cases gracefully
5. ✅ Maintain conversation context across multiple turns

### 🎓 Tutorial Series Progress:
- ✅ **Tutorial 1: Hello Spark** - Basic nodes
- ✅ **Tutorial 2: Batch Processing** - Parallel execution  
- ✅ **Tutorial 3: Simple Flows** - Graph basics (you'll see this next!)
- ✅ **Tutorial 4: Your First AI Agent** - *You are here!* 🎯
- ➡️ **Tutorial 5: Conditional Routing** - Decision making

### 🌟 Pro Tips:

- **Start with EchoModel** for testing without API costs
- **Use system prompts** to define clear agent behavior and constraints
- **Test with different models** to find the best balance of cost and performance
- **Handle errors gracefully** - agents should provide helpful feedback even when things go wrong
- **Think in conversations** - design agents to handle multi-turn interactions naturally

### 🎯 Challenge Before Next Tutorial:

Create a "Story Generator" agent that:
1. Accepts a genre and a story prompt
2. Generates a creative 3-sentence story
3. Maintains a consistent tone for the genre
4. Can continue the story if the user asks for "more"

This will prepare you well for working with conditional logic in the next tutorial!

---

**Happy building with Spark!** 🚀

Have questions or feedback? Check the Spark documentation or open an issue on GitHub.